In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
# EXPERIMENT SETUP
NUM_ITERATIONS = 20
DATASET_SIZE = 25
DATASET_CAT = ['A', 'CH', 'CR', 'J', 'LAW', 'LTD', 'TER', 'USE']

In [3]:
import sys
from pathlib import Path

# Add the LangChain directory to sys.path
ail_path = Path().resolve().parent
if str(ail_path) not in sys.path:
    sys.path.append(str(ail_path))


In [4]:
from groq import Groq
import pandas as pd
import tools.paraphrase
from tools.reformat import Reformat
from tools.score_complete import score_prompt, score_prompt_01, score_prompt_11
import tools.score_complete
import tools.ShortenTool
import tools.ExampleTool
import tools.jump_iteration
from tools.output_pipes import MultiLogger
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
import json
import re
import io
import os
import sys
import nltk
import time

nltk.download('punkt')

' - Start with "yes" or "no"\n - Justify response in no more than 50 words'


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sandradening/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sandradening/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sandradening/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sandradening/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sandradening/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
train_doc_ids = pd.read_csv('../../dataset/claudette_train_merged.tsv', sep='\t')['document'].unique()
val_doc_ids = pd.read_csv('../../dataset/claudette_val_merged.tsv', sep='\t')['document'].unique()
test_doc_ids = pd.read_csv('../../dataset/claudette_test_merged.tsv', sep='\t')['document'].unique()

df = pd.read_csv('../../dataset/claudette_all_merged.tsv', sep='\t').rename(columns={"file_name": "document"})
df_train = df.loc[df['document'].isin(train_doc_ids)]
#df_train_neg = df_train.loc[df_train['label'] == 0]
#df_train_pos = df_train.loc[df_train['label'] == 1]
df_val = df.loc[df['document'].isin(val_doc_ids)]
df_test = df.loc[df['document'].isin(test_doc_ids)]

In [6]:
#df_train_pos_per_cat = {}
for category in DATASET_CAT:
    df_train.loc[:, category] = df_train.loc[:, 'label_type'].apply(lambda cats: int(category.upper() in cats))
    #df_train.loc[:, category] = df_train_pos.loc[:, 'label_type'].apply(lambda cats: int(category.upper() in cats))
    #df_train_pos_per_cat[category] = df_train.loc[df_train[category] == 1]

/var/folders/nd/xc3t3y5n57jcb763d6d9rksw0000gn/T/ipykernel_56035/95851583.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[:, category] = df_train.loc[:, 'label_type'].apply(lambda cats: int(category.upper() in cats))
/var/folders/nd/xc3t3y5n57jcb763d6d9rksw0000gn/T/ipykernel_56035/95851583.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[:, category] = df_train.loc[:, 'label_type'].apply(lambda cats: int(category.upper() in cats))
/var/folders/nd/xc3t3y5n57jcb763d6d9rksw0000

In [7]:
def make_answer_instruction(n_words = 50):
    return f'Start your answer with "yes" or "no" and then justify your response in no more than {n_words} words.' 

In [8]:
def sample_equal_distribution(df, categories, sample_size, seed=123):
    
    per_category=int(sample_size/(len(categories)+1))
    sample_df = None

    for cat in categories:
        temp_df = df[df[cat] == 1].sample(n=per_category, random_state=seed)
        if type(sample_df) != None:
            sample_df = pd.concat([sample_df, temp_df])
        else:
            sample_df=temp_df

    temp_df = df[(df[categories] == 0).all(axis=1)].sample(n=int(sample_size/len(categories)), random_state=seed)
    sample_df = pd.concat([sample_df, temp_df])

    return sample_df

In [9]:
df_train_short = sample_equal_distribution(df_train, DATASET_CAT, DATASET_SIZE)

In [10]:
paraphrase = tools.paraphrase.ParaphrasePegasus()
shorten = tools.ShortenTool.ShortenTool()
reformat = tools.reformat.Reformat()


In [11]:
def run_prompt_optimization(intro:str, initial_prompt, df, df_short, model, num_iterations, category, examples):
    """Optimizes the prompt using an LLM to choose the best edit operation."""
    ##For this.. online: <current clause, > <Legal Description, > <Examples, >
    #optimization_history =  [{'iteration': 1, "prompt": part_of_prompt_to_modify, "score": 80, "edit": "paraphrase"}]
    print("\n ################################")
    print("Optimizing for category: ", category)

    optimization_history = []  # Initialize as an empty list attribute
    part_of_prompt_to_modify = initial_prompt
    iteration = 0
    while iteration < num_iterations:
    #for iteration in range(num_iterations):  # Iterate for the specified number of times
        # 1. Construct LLM Prompt
        if iteration == 0:
            # Special prompt for the first iteration (no history)
            len_pos = len(examples.added_examples["positive"])
            len_neg = len(examples.added_examples["negative"])
            score_value, _ = score_prompt(intro, examples.added_examples['positive'], examples.added_examples['negative'], part_of_prompt_to_modify, initial_prompt, df_short, category, model, {'p':1, 'r':0, 'l':0}) 
            optimization_history.append(
                {"iteration": 0, "prompt": part_of_prompt_to_modify, "edit": "inital - no edit", "positive_examples_count": len_pos, "negative_examples_count": len_neg, "score": score_value}  # Log 1-based iteration
            )
    
            llm_prompt = (
                "You are tasked with optimizing a prompt which is used to conduct a classification task with another LLM on a test set. The classification task is to classify legal clauses either as fair or unfair clauses of different legal unfairness categories by doing it via one vs. all classification. An unfair clause is either in a specific unfairness category or not (fair or another unfairness category)."
                "The classification result using the prompt is scored based on recall, focusing on capturing as many true positive unfair clauses as possible. The goal is to achieve a high recall score while maintaining the specificity of the model’s classification."
                f"The optimization process is done within 20 iterations. The final iteration is expected to provide the optimal prompt. In every iteration, you can choose from a list of possible actions to alter the <part_of_prompt_to_modify>. Please ensure you achieve an optimal prompt by iteration 20. This is iteration {iteration + 1}. \n"
                "The optimized prompt will be later used on another bigger unseen test set to test generalized performance. It is crucial that the classification output of the LLM starts with 'yes' for positive (unfair) classifications and 'no' for negative (fair or other category) classifications, as only the first word will determine true/false positives/negatives."
                "This is the structure of the full prompt given to the classification LLM: <intro><current_clause><legal_description><examples><part_of_prompt_to_modify>."
                "In the following, the single parts of the full prompt are described:"
                f"<intro> is: {intro} \n"
                "<current_clause>: the variable clause to be classified.\n"
                "<legal_description>: a question aiding in classifying clauses, consistent across all clauses per unfairness category.\n"
                "<examples>: Optional. Includes example clauses representing either a positive or negative clause. It is possible that both negative and positive examples are present.\n"
                f"<part_of_prompt_to_modify>: which is currently {part_of_prompt_to_modify}, and only this part should be modified in order to optimize performance.\n"
                
                "Your optimization should emphasize achieving high recall, which means capturing as many true positives as possible. Since the classification depends on the first word, ensure that the output begins with 'yes' for positives and 'no' for negatives."

                "In every iteration, choose from the following actions to improve the prompt. Aim for high recall, ensuring the prompt captures the maximum number of true positives."

                "The history contains the following information:"
                "iteration: Number of iteration.\n"
                "prompt: The used <part_of_prompt_to_modify> optimized using the actions\n"
                "edit: The action used in the respective iteration.\n"
                "score: The recall score of the prompt, representing the number of true positives correctly identified.\n"
                "relative_improvement: The relative improvement in percent compared to the previous iteration.\n"
                "token_len: Token length of the prompt given to the classification LLM.\n"
    
                "If an action does not lead to a higher score after two iterations, try a different action. If the score plateaus, return to a previous iteration by taking the respective action 'jump_back_to_iteration <number of iteration>'. Try different actions to gauge which leads to improved scores."
                "Select the most suitable action to enhance the prompt and achieve the highest recall score by iteration 20:"
                "- shorten: Shortens the prompt by removing stopwords. The output does not change if used twice consecutively.\n"
                "- reword_prompt: Reword the <part_of_prompt_to_modify> using the 'PegasusTokenizer'.\n"
                "- to_bulletpoints: Formats the <part_of_prompt_to_modify> into bullet points. The output does not change if used twice consecutively.\n"
                "- add_positive_example: Include a positive example. A positive example is a clause that belongs to the same category and should be classified as 'yes'.\n"
                "- add_negative_example: Include a negative example. A negative example is a clause that belongs to a different category and should be classified as 'no'.\n"
                "- remove_positive_example: Remove one of the positive examples previously added with the add_positive_example action. A positive example is a clause that belongs to the same category and should be classified as 'yes'.\n"
                "- remove_negative_example: Remove one of the negative examples previously added with the add_negative_example action. A negative example is a clause that belongs to a different category and should be classified as 'no'.\n"
                   
                "Your output should be formatted as: <chosen_action> because: <reasoning>"
            )
        else:
            # Prompt for subsequent iterations (with history)
            llm_prompt = (
                "You are tasked with optimizing a prompt which is used to conduct a classification task with another LLM on a test set. The classification task is to classify legal clauses either as fair or unfair clauses of different legal unfairness categories by doing it via one vs. all classification. An unfair clause is either in a specific unfairness category or not (fair or another unfairness category)."
                "The classification result using the prompt is scored based on recall, focusing on capturing as many true positive unfair clauses as possible. The goal is to achieve a high recall score while maintaining the specificity of the model’s classification."
                f"The optimization process is done within 20 iterations. The final iteration is expected to provide the optimal prompt. In every iteration, you can choose from a list of possible actions to alter the <part_of_prompt_to_modify>. Please ensure you achieve an optimal prompt by iteration 20. This is iteration {iteration + 1}. \n"
                "The optimized prompt will be later used on another bigger unseen test set to test generalized performance. It is crucial that the classification output of the LLM starts with 'yes' for positive (unfair) classifications and 'no' for negative (fair or other category) classifications, as only the first word will determine true/false positives/negatives."
                "This is the structure of the full prompt given to the classification LLM: <intro><current_clause><legal_description><examples><part_of_prompt_to_modify>."
                "In the following, the single parts of the full prompt are described:"
                f"<intro> is: {intro} \n"
                "<current_clause>: the variable clause to be classified.\n"
                "<legal_description>: a question aiding in classifying clauses, consistent across all clauses per unfairness category.\n"
                "<examples>: Optional. Includes example clauses representing either a positive or negative clause. It is possible that both negative and positive examples are present.\n"
                f"<part_of_prompt_to_modify>: which is currently {part_of_prompt_to_modify}, and only this part should be modified in order to optimize performance.\n"
                
                "Your optimization should emphasize achieving high recall, which means capturing as many true positives as possible. Since the classification depends on the first word, ensure that the output begins with 'yes' for positives and 'no' for negatives."

                "In every iteration, choose from the following actions to improve the prompt. Aim for high recall, ensuring the prompt captures the maximum number of true positives."

                "The history contains the following information:"
                "iteration: Number of iteration.\n"
                "prompt: The used <part_of_prompt_to_modify> optimized using the actions\n"
                "edit: The action used in the respective iteration.\n"
                "score: The recall score of the prompt, representing the number of true positives correctly identified.\n"
                "relative_improvement: The relative improvement in percent compared to the previous iteration.\n"
                "token_len: Token length of the prompt given to the classification LLM.\n"
    
                "If an action does not lead to a higher score after two iterations, try a different action. If the score plateaus, return to a previous iteration by taking the respective action 'jump_back_to_iteration <number of iteration>'. Try different actions to gauge which leads to improved scores."
                "Select the most suitable action to enhance the prompt and achieve the highest recall score by iteration 20:"
                "- shorten: Shortens the prompt by removing stopwords. The output does not change if used twice consecutively.\n"
                "- reword_prompt: Reword the <part_of_prompt_to_modify> using the 'PegasusTokenizer'.\n"
                "- to_bulletpoints: Formats the <part_of_prompt_to_modify> into bullet points. The output does not change if used twice consecutively.\n"
                "- add_positive_example: Include a positive example. A positive example is a clause that belongs to the same category and should be classified as 'yes'.\n"
                "- add_negative_example: Include a negative example. A negative example is a clause that belongs to a different category and should be classified as 'no'.\n"
                "- remove_positive_example: Remove one of the positive examples previously added with the add_positive_example action. A positive example is a clause that belongs to the same category and should be classified as 'yes'.\n"
                "- remove_negative_example: Remove one of the negative examples previously added with the add_negative_example action. A negative example is a clause that belongs to a different category and should be classified as 'no'.\n"
                "- jump_back_to_iteration: Revert to a previous iteration's <part_of_prompt_to_modify> and number of examples if earlier actions led to better performance. Format: 'jump_back_to_iteration <number>' because: <reasoning>'\n"
                
                "Your output should be formatted as: <chosen_action> because: <reasoning>"

                #"You are tasked with optimizing a prompt which is used to conduct a classification task with another LLM on a test set. The classification task is to classify legal clauses either fair or unfair clauses of different legal unfairness categories by doing it via one vs. all classification. An unfair clause is either in a specific unfairness category or not (fair or another unfairness category)." 
                #"The classification result using the prompt is scored on a scale of 0 - 100 (F1 Score times 100). Your goal is to find the best prompt and get a score as high as possible."
                #f"The optimization process is done within {num_iterations} iterations the final iteration is expected to provide the optimal prompt. In every iteration, you can choose from a list of possible actions to alter the <part_of_prompt_to_modify>. Please make sure you achieve an optimal prompt until iteration {num_iterations}. This is iteration {iteration + 1}. \n "
                #"The optimized prompt will be later used on another bigger unseen test set to test generalized performance."
                #"This is the structure of the full prompt given to the classification LLM: <intro><current_clause><legal_description><examples><part_of_prompt_to_modify> "
                #"In the following, the single parts of the full prompt are described:"
                #f"<intro> is: {intro} \n"
                #"<current_clause> is the variable clause to be classified.\n"
                #"<legal_description> is a question helping to classify clauses and is the same for all clauses per unfairness category.\n"
                #"<examples> is optional this is an example clause representing either a positive or negative clause. It is possible that both negative and positive are present.\n"
                #f"<part_of_prompt_to_modify> is {part_of_prompt_to_modify}, only modify this part \n"
                #"Your optimization should consider how the classification performance is measured: The output of the classification LLM is classified positive if the output starts with 'yes' and negative if the output starts with 'no', so only the first word will be considered when determining true / false positives / negatives.\n"
                #"To optimize the prompt, you can use various actions. "
                #f"This is the history of all conducted operations until now: {optimization_history}\n"
                #"The history contains the following information: \n" 
                #"iteration: Number of iteration.\n"
                #"prompt: The used <part_of_prompt_to_modify> which is the part of the prompt which you can optimize using the actions.\n"
                #"edit: The action that is used in the respective iteration.\n"
                #"score: The achieved score of the prompt which was edited using the respective action.\n"
                #"relative_improvement: The relative improvement in percent compared to the previous iteration.\n"
                #"token_len: token length of the prompt given to the classification LLM.\n"
                #"If an action did not lead to a higher score after two iterations, try a different action. "
                #"If you find that the score plateaus, it is always possible to return to a previous iteration by taking the respective action 'jump_back_to_iteration <number of iteration>'. "
                #"Try different actions to get a feeling which leads to a improving scores. \n"
                #f"These are the actions you can choose from - use the most suitable action to further improve the prompt and achieve the highest score by iteration {num_iterations}:\n"
                #"- shorten: Shortens the prompt by removing stopwords from the prompt. The output does not change if used twice in a row.\n"
                #"- reword_prompt: Reword the <part_of_prompt_to_modify> using the 'PegasusTokenizer'.\n"
                #"- to_bulletpoints: Formates the <part_of_prompt_to_modify> into bullet points. The output does not change if used twice in a row.\n"
                #"- add_positive_example: Include a positive example. A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                #"- add_negative_example: Include a negative example. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                #"- remove_positive_example: Remove one of the positive examples previously added with the add_positive_example action.  A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                #"- remove_negative_example: Remove one of the negative examples previously added with the add_negative_example action. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                #"- jump_back_to_iteration: Jump back to the <part_of_prompt_to_modify> and number of positive and negative examples of any previous iteration. If you realize previous actions led to a decrease in performance, you can always return to an iteration step that has been more promising. If you choose this action, your output should be of form: jump_back_to_iteration <number> because: <reasoning>'.\n"
                    
                #"Your output should be of form: <chosen_action> because: <reasoning>"
            )

        print(f"\nThis is iteration {iteration + 1}")
        try:
            # 2. Get LLM's Decision
            message = HumanMessage(content=llm_prompt)
            response = model.invoke([message])
            print(f"Complete response: {response}")
            chosen_action = response.content.lower().split()[0]
            #print("Response: " + response)
            print("Chosen Action: " + chosen_action)
            print("Optimizing the current <part_of_prompt_to_modify>: \n\t", part_of_prompt_to_modify)

            #if iteration == 1:
            #    chosen_action = "jump_back_to_iteration"
            #    iteration_number = 0
            
            # 3. Apply Chosen Action
            for i in range(2):
                if chosen_action == "shorten":
                    part_of_prompt_to_modify = shorten.shorten_prompt(part_of_prompt_to_modify)
                    break
                elif chosen_action == "add_positive_example":
                    examples.add_positive_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "add_negative_example":
                    examples.add_negative_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "remove_positive_example":
                    examples.remove_positive_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "remove_negative_example":
                    examples.remove_negative_example(part_of_prompt_to_modify,category)
                    break            
                elif chosen_action == "reword_prompt":
                    part_of_prompt_to_modify = paraphrase.paraphrase_pegasus(part_of_prompt_to_modify)
                    break
                elif chosen_action == "to_bulletpoints":
                    part_of_prompt_to_modify = reformat.reformat(part_of_prompt_to_modify)
                    break
                elif chosen_action == "jump_back_to_iteration":
                    match = re.search(r'jump_back_to_iteration (\d+)', response.content.lower())
                    if match:
                        iteration_number = int(match.group(1))
                    print(chosen_action, iteration_number)
                    part_of_prompt_to_modify = jump_iteration.jump_back_to_iteration(iteration_number, optimization_history, category)
                    print(part_of_prompt_to_modify)
                    break
                #elif chosen_action == "grammar_adjustment":
                #    grammarAdjustment = GrammarAdjustment()
                #    part_of_prompt_to_modify = grammarAdjustment.grammar_adjustment(current_prompt)
                else:
                    keywords = [
                        "shorten", 
                        "add_positive_example", 
                        "add_negative_example", 
                        "remove_positive_example", 
                        "remove_negative_example", 
                        "reword_prompt",  
                        "to_bulletpoints",
                        "jump_back_to_iteration"
                    ]
                    pattern = r'jump_back_to_iteration (\d+)'
                    match = re.search(pattern, response.content.lower())
                    if match:
                        chosen_action="jump_back_to_iteration"
                    else:
                        for keyword in keywords:
                            if keyword in response.content.lower():
                                chosen_action = keyword 
                                break
                            else:
                                chosen_action = ""
                        if chosen_action == "":        
                            raise ValueError(f"Invalid action chosen by LLM: {chosen_action}")
                    print(f"Invalid chosen action. New chosen action after error handling is: {chosen_action}")

            # 4. Score and Log (using self.optimization_history)
            score_value, mean_token_len = score_prompt(intro, examples.added_examples['positive'], examples.added_examples['negative'], part_of_prompt_to_modify, initial_prompt, df_short, category, model, {'p':1, 'r':0, 'l':0}) # + legal description, df_test, category, intro 
            len_pos = len(examples.added_examples["positive"])
            len_neg = len(examples.added_examples["negative"])
            relative_improvment = str(((score_value/optimization_history[iteration]['score'])-1)*100) + '%'
            optimization_history.append(
                {"iteration": iteration + 1, "prompt": part_of_prompt_to_modify, "edit": chosen_action, "positive_examples_count": len_pos, "negative_examples_count": len_neg, "score": score_value, "relative_improvement": relative_improvment, "token_len": mean_token_len}  # Log 1-based iteration
            )
            
            if score_value >= 100:
                print("Early stopping: Recall has reached 100%")
                return optimization_history
            
            iteration += 1
        except Exception as e:
            print(f"Error during optimization  {e}")
            print("Retrying the iteration...")
            #break

    return optimization_history


In [12]:
def run_prompt_optimization_CH(intro:str, initial_prompt, df, df_short, model, num_iterations, category, examples):
    """Optimizes the prompt using an LLM to choose the best edit operation."""
    ##For this.. online: <current clause, > <Legal Description, > <Examples, >
    #optimization_history =  [{'iteration': 1, "prompt": part_of_prompt_to_modify, "score": 80, "edit": "paraphrase"}]
    print("\n ################################")
    print("Optimizing for category: ", category)

    optimization_history = []  # Initialize as an empty list attribute
    part_of_prompt_to_modify = initial_prompt
    iteration = 0
    while iteration < num_iterations:
    #for iteration in range(num_iterations):  # Iterate for the specified number of times
        # 1. Construct LLM Prompt
        if iteration == 0:
            # Special prompt for the first iteration (no history)
            len_pos = len(examples.added_examples["positive"])
            len_neg = len(examples.added_examples["negative"])
            score_value, _ = score_prompt(intro, examples.added_examples['positive'], examples.added_examples['negative'], part_of_prompt_to_modify, initial_prompt, df_short, category, model, {'p':1, 'r':0, 'l':0}) 
            optimization_history.append(
                {"iteration": 0, "prompt": part_of_prompt_to_modify, "edit": "inital - no edit", "positive_examples_count": len_pos, "negative_examples_count": len_neg, "score": score_value}  # Log 1-based iteration
            )
    
            llm_prompt = (
                f"You are tasked with optimizing a prompt used to conduct a classification task with another LLM. " 
                "The classification result using the prompt is scored on a scale of 0 - 100. Your goal is to find the best prompt and get a score as high as possible. "
                f"The optimization process is done within {num_iterations} iterations the final iteration is expected to provide the optimal prompt. In every iteration, you can choose from a list of possible actions to alter the <part_of_prompt_to_modify>. Please make sure you achieve an optimal prompt until iteration {num_iterations}. This is iteration {iteration + 1}. \n "
                "This is the structure of the full prompt given to the classification LLM: <intro><current_clause><legal_description><examples><part_of_prompt_to_modify> "
                "In the following, the single parts of the full prompt are described:"
                f"<intro> is: {intro} \n"
                "<current_clause> is the variable clause to be classified.\n"
                f"<legal_description> is a question helping to classify clauses.\n"
                "<examples> is optional this is an example clause representing either a positive or negative clause. It is possible that both negative and positive are present.\n"
                f"<part_of_prompt_to_modify> is {part_of_prompt_to_modify}, only modify this part \n"
                "The output of the classification LLM is classified positive if the output starts with yes and negative if the output starts with no.\n"
                "To optimize the prompt, you can use various actions. "
                f"This is the history of all conducted operations until now: {optimization_history}\n"
                "The history contains the following information: \n" 
                "iteration: Number of iterations.\n"
                "prompt: The used <part_of_prompt_to_modify> so the optimized part of the prompt.\n"
                "edit: The action that is used in the regrading iteration.\n"
                "score: The achieved score of the prompt edited by the respective action.\n"
                "relative_improvement: The relative improvement in percent compared to the previous iteration.\n"
                "token_len: token length of the prompt given to the classification LLM.\n"
                "If an action did not lead to a higher score after two iterations, try a different action. "
                "If you find that the score plateaus, it is always possible to return to a previous iteration by taking the respective action. "
                "Try different actions to get a feeling which leads to a higher score. \n"
                " These are the actions you can choose from. Use the most suitable action to improve the prompt further :\n"
                "- action_1: Shortens the prompt by removing stopwords from the prompt. The output does not change if used twice in a row.\n"
                "- action_6: Reword the <part_of_prompt_to_modify>. \n"
                "- action_7: Formates the <part_of_prompt_to_modify> into bullet points. The output does not change if used twice in a row.\n"
                "- action_2: Include a positive example. A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                "- action_3: Include a negative example. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                "- action_4: Remove one of the positive examples previously added with the add_positive_example action.  A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                "- action_5: Remove one of the negative examples previously added with the add_negative_example action. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                "Your output should be of form: <chosen_action> because: <reasoning>"
            )
        else:
            # Prompt for subsequent iterations (with history)
            llm_prompt = (
                f"You are tasked with optimizing a prompt used to conduct a classification task with another LLM. " 
                "The classification result using the prompt is scored on a scale of 0 - 100. Your goal is to find the best prompt and get a score as high as possible. "
                f"The optimization process is done within {num_iterations} iterations the final iteration is expected to provide the optimal prompt. In every iteration, you can choose from a list of possible actions to alter the <part_of_prompt_to_modify>. Please make sure you achieve an optimal prompt until iteration {num_iterations}. This is iteration {iteration + 1}. \n "
                "This is the structure of the full prompt given to the classification LLM: <intro><current_clause><legal_description><examples><part_of_prompt_to_modify> "
                "In the following, the single parts of the full prompt are described:"
                f"<intro> is: {intro} \n"
                "<current_clause> is the variable clause to be classified.\n"
                f"<legal_description> is a question helping to classify clauses.\n"
                "<examples> is optional this is an example clause representing either a positive or negative clause. It is possible that both negative and positive are present.\n"
                f"<part_of_prompt_to_modify> is {part_of_prompt_to_modify}, only modify this part \n"
                "The output of the classification LLM is classified positive if the output starts with yes and negative if the output starts with no.\n"
                "To optimize the prompt, you can use various actions. "
                f"This is the history of all conducted operations until now: {optimization_history}\n"
                "The history contains the following information: \n" 
                "iteration: Number of iterations.\n"
                "prompt: The used <part_of_prompt_to_modify> so the optimized part of the prompt.\n"
                "edit: The action that is used in the regrading iteration.\n"
                "score: The achieved score of the prompt edited by the respective action.\n"
                "relative_improvement: The relative improvement in percent compared to the previous iteration.\n"
                "token_len: token length of the prompt given to the classification LLM.\n"
                "If an action did not lead to a higher score after two iterations, try a different action. "
                "If you find that the score plateaus, it is always possible to return to a previous iteration by taking the respective action. "
                "Try different actions to get a feeling which leads to a higher score. \n"
                " These are the actions you can choose from. Use the most suitable action to improve the prompt further :\n"
                "- action_1: Shortens the prompt by removing stopwords from the prompt. The output does not change if used twice in a row.\n"
                "- action_6: Reword the <part_of_prompt_to_modify>. \n"
                "- action_7: Formates the <part_of_prompt_to_modify> into bullet points. The output does not change if used twice in a row.\n"
                "- action_2: Include a positive example. A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                "- action_3: Include a negative example. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                "- action_4: Remove one of the positive examples previously added with the add_positive_example action.  A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                "- action_5: Remove one of the negative examples previously added with the add_negative_example action. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                "- action_8: Jump back to a previous iteration. If you realize previous actions led to a decrease in performance, you can always return to an iteration step that has been more promising. If you choose this action, your output should be of form: jump_back_to_iteration <number> because: <reasoning>'.\n"
                    
                #"- grammar_adjustment: Fix any grammatical errors in the prompt.\n"
                "Your output should be of form: <chosen_action> because: <reasoning>"
            )

        print(f"\nThis is iteration {iteration + 1}")
        try:
            # 2. Get LLM's Decision
            message = HumanMessage(content=llm_prompt)
            response = model.invoke([message])
            print(f"Complete response: {response}")
            chosen_action = response.content.lower().split()[0]
            #print("Response: " + response)
            print("Chosen Action: " + chosen_action)

            #if iteration == 1:
            #    chosen_action = "jump_back_to_iteration"
            #    iteration_number = 0
            
            # 3. Apply Chosen Action
            for i in range(2):
                if chosen_action == "action_1":
                    part_of_prompt_to_modify = shorten.shorten_prompt(part_of_prompt_to_modify)
                    break
                elif chosen_action == "action_2":
                    examples.add_positive_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "action_3":
                    examples.add_negative_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "action_4":
                    examples.remove_positive_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "action_5":
                    examples.remove_negative_example(part_of_prompt_to_modify,category)
                    break            
                elif chosen_action == "action_6":
                    part_of_prompt_to_modify = paraphrase.paraphrase_pegasus(part_of_prompt_to_modify)
                    break
                elif chosen_action == "action_7":
                    part_of_prompt_to_modify = reformat.reformat(part_of_prompt_to_modify)
                    break
                elif chosen_action == "action_8":
                    match = re.search(r'jump_back_to_iteration (\d+)', response.content.lower())
                    if match:
                        iteration_number = int(match.group(1))
                    print(chosen_action, iteration_number)
                    part_of_prompt_to_modify = jump_iteration.jump_back_to_iteration(iteration_number, optimization_history, category)
                    print(part_of_prompt_to_modify)
                    break
                #elif chosen_action == "grammar_adjustment":
                #    grammarAdjustment = GrammarAdjustment()
                #    part_of_prompt_to_modify = grammarAdjustment.grammar_adjustment(current_prompt)
                else:
                    keywords = [
                        "action_1", 
                        "action_2", 
                        "action_3", 
                        "action_4", 
                        "action_5", 
                        "action_6",  
                        "action_7",
                        "action_8"
                    ]
                    pattern = r'jump_back_to_iteration (\d+)'
                    match = re.search(pattern, response.content.lower())
                    if match:
                        chosen_action="jump_back_to_iteration"
                    else:
                        for keyword in keywords:
                            if keyword in response.content.lower():
                                chosen_action = keyword 
                                break
                            else:
                                chosen_action = ""
                        if chosen_action == "":        
                            raise ValueError(f"Invalid action chosen by LLM: {chosen_action}")
                    print(f"Invalid chosen action. New chosen action after error handling is: {chosen_action}")

            # 4. Score and Log (using self.optimization_history)
            score_value, mean_token_len = score_prompt(intro, examples.added_examples['positive'], examples.added_examples['negative'], part_of_prompt_to_modify, initial_prompt, df_short, category, model, {'p':1, 'r':0, 'l':0}) # + legal description, df_test, category, intro 
            len_pos = len(examples.added_examples["positive"])
            len_neg = len(examples.added_examples["negative"])
            relative_improvment = str(((score_value/optimization_history[iteration]['score'])-1)*100) + '%'
            optimization_history.append(
                {"iteration": iteration + 1, "prompt": part_of_prompt_to_modify, "edit": chosen_action, "positive_examples_count": len_pos, "negative_examples_count": len_neg, "score": score_value, "relative_improvement": relative_improvment, "token_len": mean_token_len}  # Log 1-based iteration
            )
            iteration += 1
        except Exception as e:
            print(f"Error during optimization  {e}")
            print("Retrying the iteration...")
            #break

    return optimization_history

In [13]:
def run_prompt_optimization(intro:str, initial_prompt, df, df_short, model, num_iterations, category, examples):
    """Optimizes the prompt using an LLM to choose the best edit operation."""
    ##For this.. online: <current clause, > <Legal Description, > <Examples, >
    #optimization_history =  [{'iteration': 1, "prompt": part_of_prompt_to_modify, "score": 80, "edit": "paraphrase"}]
    print("\n ################################")
    print("Optimizing for category: ", category)

    optimization_history = []  # Initialize as an empty list attribute
    part_of_prompt_to_modify = initial_prompt
    iteration = 0
    while iteration < num_iterations:
    #for iteration in range(num_iterations):  # Iterate for the specified number of times
        # 1. Construct LLM Prompt
        if iteration == 0:
            # Special prompt for the first iteration (no history)
            len_pos = len(examples.added_examples["positive"])
            len_neg = len(examples.added_examples["negative"])
            score_value, _ = score_prompt(intro, examples.added_examples['positive'], examples.added_examples['negative'], part_of_prompt_to_modify, initial_prompt, df_short, category, model, {'p':1, 'r':0, 'l':0}) 
            optimization_history.append(
                {"iteration": 0, "prompt": part_of_prompt_to_modify, "edit": "inital - no edit", "positive_examples_count": len_pos, "negative_examples_count": len_neg, "score": score_value}  # Log 1-based iteration
            )
    
            llm_prompt = (
                f"You are tasked with optimizing a prompt. This is the inital iteration (iteration 0).\n" 
                "The prompt is scored in a scale of 0 - 100. Your goal is to get a score that is high as possible."
                "Full Prompt: <intro><current_clause><legal_description><examples><part_of_prompt_to_modify>"
                f"<intro> is: {intro}"
                "<current_clause> is the variable clause to be classified."
                f"<legal_description> is: Question helping to classify clauses"
                "<examples> is: optional, example clauses that are either positive or negative to the classification"
                f"<part_of_prompt_to_modify> is {part_of_prompt_to_modify}, you will only modify this part\n\n"
                f"Your starting point: {optimization_history}\n\n"
                "To optimize the prompt you can use various actions. These actions alter the prompt."
                #"To optimize the prompt you can only use one action. "
                "Choose the most suitable action to improve the prompt:\n"
                "- shorten: Shortens the prompt by removing stopwords from the prompt.\n"
                "- add_positive_example: Include a positive example.\n"
                "- add_negative_example: Include a negative example.\n"
                "- reword_prompt: Reword the <part_of_prompt_to_modify>.\n"
                "- to_bulletpoints: Formates the <part_of_prompt_to_modify> into bullet points. The output does not change if used twice in a row.\n"
                #"- grammar_adjustment: Fix any grammatical errors in the prompt.\n"
                #"- jump_back_to_iteration: Jump back to a specific iteration. Use the format 'jump_back_to_iteration <number> because: <reasoning>'.\n"
                "Your output should be of form: <chosen_action> because: <reasoning>"
            )
        else:
            # Prompt for subsequent iterations (with history)
            llm_prompt = (
                f"You are tasked with optimizing a prompt used to conduct a classification task with another LLM. " 
                "The classification result using the prompt is scored on a scale of 0 - 100. Your goal is to find the best prompt and get a score as high as possible. "
                f"The optimization process is done within {num_iterations} iterations the final iteration is expected to provide the optimal prompt. In every iteration, you can choose from a list of possible actions to alter the <part_of_prompt_to_modify>. Please make sure you achieve an optimal prompt until iteration {num_iterations}. This is iteration {iteration + 1}. \n "
                "This is the structure of the full prompt given to the classification LLM: <intro><current_clause><legal_description><examples><part_of_prompt_to_modify> "
                "In the following, the single parts of the full prompt are described:"
                f"<intro> is: {intro} \n"
                "<current_clause> is the variable clause to be classified.\n"
                f"<legal_description> is a question helping to classify clauses.\n"
                "<examples> is optional this is an example clause representing either a positive or negative clause. It is possible that both negative and positive are present.\n"
                f"<part_of_prompt_to_modify> is {part_of_prompt_to_modify}, only modify this part \n"
                "The output of the classification LLM is classified positive if the output starts with yes and negative if the output starts with no.\n"
                "To optimize the prompt, you can use various actions. "
                f"This is the history of all conducted operations until now: {optimization_history}\n"
                "The history contains the following information: \n" 
                "iteration: Number of iterations.\n"
                "prompt: The used <part_of_prompt_to_modify> so the optimized part of the prompt.\n"
                "edit: The action that is used in the regrading iteration.\n"
                "score: The achieved score of the prompt edited by the respective action.\n"
                "relative_improvement: The relative improvement in percent compared to the previous iteration.\n"
                "token_len: token length of the prompt given to the classification LLM.\n"
                "If an action did not lead to a higher score after two iterations, try a different action. "
                "If you find that the score plateaus, it is always possible to return to a previous iteration by taking the respective action. "
                "Try different actions to get a feeling which leads to a higher score. \n"
                " These are the actions you can choose from. Use the most suitable action to improve the prompt further :\n"
                "- shorten: Shortens the prompt by removing stopwords from the prompt. The output does not change if used twice in a row.\n"
                "- reword_prompt: Reword the <part_of_prompt_to_modify>. \n"
                "- to_bulletpoints: Formates the <part_of_prompt_to_modify> into bullet points. The output does not change if used twice in a row.\n"
                "- add_positive_example: Include a positive example. A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                "- add_negative_example: Include a negative example. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                "- remove_positive_example: Remove one of the positive examples previously added with the add_positive_example action.  A positive example is a clause that belongs to the same category and should be classified as true (a category of several unfairness categories).\n"
                "- remove_negative_example: Remove one of the negative examples previously added with the add_negative_example action. A negative example is a clause that belongs to a different category and should be classified as false (a category of several unfairness categories).\n"
                "- jump_back_to_iteration: Jump back to a previous iteration. If you realize previous actions led to a decrease in performance, you can always return to an iteration step that has been more promising. If you choose this action, your output should be of form: jump_back_to_iteration <number> because: <reasoning>'.\n"
                
                #"- grammar_adjustment: Fix any grammatical errors in the prompt.\n"
                "Your output should be of form: <chosen_action> because: <reasoning>"
            )

        print(f"\nThis is iteration {iteration + 1}")
        try:
            # 2. Get LLM's Decision
            message = HumanMessage(content=llm_prompt)
            response = model.invoke([message])
            print(f"Complete response: {response}")
            chosen_action = response.content.lower().split()[0]
            #print("Response: " + response)
            print("Chosen Action: " + chosen_action)

            #if iteration == 1:
            #    chosen_action = "jump_back_to_iteration"
            #    iteration_number = 0
            
            # 3. Apply Chosen Action
            for i in range(2):
                if chosen_action == "shorten":
                    part_of_prompt_to_modify = shorten.shorten_prompt(part_of_prompt_to_modify)
                    break
                elif chosen_action == "add_positive_example":
                    examples.add_positive_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "add_negative_example":
                    examples.add_negative_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "remove_positive_example":
                    examples.remove_positive_example(part_of_prompt_to_modify,category)
                    break
                elif chosen_action == "remove_negative_example":
                    examples.remove_negative_example(part_of_prompt_to_modify,category)
                    break            
                elif chosen_action == "reword_prompt":
                    part_of_prompt_to_modify = paraphrase.paraphrase_pegasus(part_of_prompt_to_modify)
                    break
                elif chosen_action == "to_bulletpoints":
                    part_of_prompt_to_modify = reformat.reformat(part_of_prompt_to_modify)
                    break
                elif chosen_action == "jump_back_to_iteration":
                    match = re.search(r'jump_back_to_iteration (\d+)', response.content.lower())
                    if match:
                        iteration_number = int(match.group(1))
                    print(chosen_action, iteration_number)
                    part_of_prompt_to_modify = jump_iteration.jump_back_to_iteration(iteration_number, optimization_history, category)
                    print(part_of_prompt_to_modify)
                    break
                #elif chosen_action == "grammar_adjustment":
                #    grammarAdjustment = GrammarAdjustment()
                #    part_of_prompt_to_modify = grammarAdjustment.grammar_adjustment(current_prompt)
                else:
                    keywords = [
                        "shorten", 
                        "add_positive_example", 
                        "add_negative_example", 
                        "remove_positive_example", 
                        "remove_negative_example", 
                        "reword_prompt",  
                        "to_bulletpoints",
                        "jump_back_to_iteration"
                    ]
                    pattern = r'jump_back_to_iteration (\d+)'
                    match = re.search(pattern, response.content.lower())
                    if match:
                        chosen_action="jump_back_to_iteration"
                    else:
                        for keyword in keywords:
                            if keyword in response.content.lower():
                                chosen_action = keyword 
                                break
                            else:
                                chosen_action = ""
                        if chosen_action == "":        
                            raise ValueError(f"Invalid action chosen by LLM: {chosen_action}")
                    print(f"Invalid chosen action. New chosen action after error handling is: {chosen_action}")

            # 4. Score and Log (using self.optimization_history)
            score_value, mean_token_len = score_prompt(intro, examples.added_examples['positive'], examples.added_examples['negative'], part_of_prompt_to_modify, initial_prompt, df_short, category, model, {'p':1, 'r':0, 'l':0}) # + legal description, df_test, category, intro 
            len_pos = len(examples.added_examples["positive"])
            len_neg = len(examples.added_examples["negative"])
            relative_improvment = str(((score_value/optimization_history[iteration]['score'])-1)*100) + '%'
            optimization_history.append(
                {"iteration": iteration + 1, "prompt": part_of_prompt_to_modify, "edit": chosen_action, "positive_examples_count": len_pos, "negative_examples_count": len_neg, "score": score_value, "relative_improvement": relative_improvment, "token_len": mean_token_len}  # Log 1-based iteration
            )
            iteration += 1
        except Exception as e:
            print(f"Error during optimization  {e}")
            print("Retrying the iteration...")
            #break

    return optimization_history

In [14]:
# Define your model (using your actual API key/setup)
model = ChatGroq(
    model_name="llama3-70b-8192",
    groq_api_key="<API KEY>",
    temperature=0
)

# Initial prompt to be optimized
optimized_prompt_dict = {}
#initial_prompt = "Consider the following online terms of service clause: 'websites & communications terms of use'. Does this clause describe an arbitration dispute resolution process that is not fully optional to the consumer? Begin your answer with 'yes' or 'no' and then justify your response in no more than 50 words. For example, consider these example clauses: <positive, if you are not a consumer in the eea , the exclusive place of jurisdiction for all disputes arising from or in connection with this agreement is san francisco county , california , or the united states district court for the northern district of california and our dispute will bedetermined under california law .'>, <negative, you are prohibited from using any services or facilities provided in connection with this service to compromise security or tamper with system resources and/or accounts .>"
unfairness_categories = ['A', 'CH', 'CR', 'J', 'LAW', 'LTD', 'TER', 'USE']
unfairness_categories_green = ['A', 'CH', 'CR', 'J']
unfairness_categories_pink = ['LAW', 'LTD', 'TER', 'USE']
unfairness_categories_CH = ['CH']


for i in range(5,8):

    print(f"\n\n++++++++++++++ This is RUN {i} of volatility analysis ++++++++++++++\n\n")

    initial_prompt = "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words."
    intro_prompt = 'Consider the following online terms of service clause:'
    #cat_results = {}
    #df_train_sample = sample_equal_distribution(df_train, unfairness_categories, 100, 123)
    #score_set_df = sample_equal_distribution(df_train, unfairness_categories, 25, 234)
    output = ''
    
    filename = f"20240808_volatility_analysis_loop_{i}.txt"
    output_dir = os.path.join('..', 'output')
    file_path = os.path.join(output_dir, filename)
    if os.path.exists(file_path):
        raise FileExistsError(f"Error: The file '{filename}' already exists in the 'output' directory. "
                                f"It is not recommended to overwrite an output file. Please specify another name.")
    
    # Create and open the file
    file_stream = open(file_path, 'w')

    
    # Buffer for capturing print output
    buffer = io.StringIO()
    
    # MultiLogger setup to write to stdout, buffer, and file
    multi_logger = MultiLogger(sys.stdout, buffer, file_stream)
    sys.stdout = multi_logger

    try:
        for cat in unfairness_categories_CH:
            examples_instance = tools.ExampleTool.ExampleTool(df_train)
            jump_iteration = tools.jump_iteration.jump_iteration(examples_instance)
            start_time = time.time()
            optimized_prompt_dict[cat] = run_prompt_optimization(intro_prompt, initial_prompt, df_train, df_train_short, model, NUM_ITERATIONS, cat, examples_instance)
            end_time = time.time()
            
            elapse_time = end_time - start_time
            elapse_minutes = int(elapse_time // 60)
            elapse_seconds = elapse_time % 60
    
            print(f"Total time of cat {cat} is {elapse_minutes} min and {elapse_seconds} sec.")
            print("Positive example list: \n", examples_instance.added_examples["positive"])
            print("\nNegative example list: \n",examples_instance.added_examples["negative"])
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
    
        print("\n\n")
        print(optimized_prompt_dict)
        # Cleanup
        sys.stdout = sys.__stdout__  # Reset stdout to default
        file_stream.close()
    
        output = buffer.getvalue()




++++++++++++++ This is RUN 5 of volatility analysis ++++++++++++++



 ################################
Optimizing for category:  CH
Start getting performance score
Start analyzing predicitions:
.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


..................
Performance score is: 11.11111111111111
Start getting length score
Lenght score is: 0

This is iteration 1
Complete response: content='reword_prompt because: The initial prompt is quite direct and concise, but rewording it could make it clearer and more specific, which might improve the score.' response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 368, 'total_tokens': 402, 'completion_time': 0.103732485, 'prompt_time': 0.040918781, 'queue_time': None, 'total_time': 0.144651266}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None} id='run-9f79bf61-3fae-4c4c-b712-50c82f660734-0' usage_metadata={'input_tokens': 368, 'output_tokens': 34, 'total_tokens': 402}
Chosen Action: reword_prompt
Starting to paraphrase the following prompt: Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words. 
 Using Pegasus:
Start getting performance score
Start analyzing

/var/folders/nd/xc3t3y5n57jcb763d6d9rksw0000gn/T/ipykernel_56035/2626945926.py:163: RuntimeWarning: divide by zero encountered in scalar divide
  relative_improvment = str(((score_value/optimization_history[iteration]['score'])-1)*100) + '%'


Complete response: content="reword_prompt because: The previous iterations didn't show significant improvement, and the score is still low. Rewording the prompt might help to clarify the task and improve the classification performance. Additionally, it's a relatively safe action that doesn't add or remove examples, which could potentially disrupt the classification process." response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 1256, 'total_tokens': 1318, 'completion_time': 0.198644871, 'prompt_time': 0.090781591, 'queue_time': None, 'total_time': 0.289426462}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None} id='run-842dbee4-48bf-412a-9728-996a7558d3a0-0' usage_metadata={'input_tokens': 1256, 'output_tokens': 62, 'total_tokens': 1318}
Chosen Action: reword_prompt
Starting to paraphrase the following prompt: Start your answer with 'yes' or 'no' and then justify it in no more than 50 words. 
 Using 

In [17]:
optimized_prompt_dict

{'CH': [{'iteration': 0,
   'prompt': "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words.",
   'edit': 'inital - no edit',
   'positive_examples_count': 0,
   'negative_examples_count': 0,
   'score': 11.11111111111111},
  {'iteration': 1,
   'prompt': "When you start your answer with 'yes' or 'no', you need to justify it in no more than 50 words.",
   'edit': 'reword_prompt',
   'positive_examples_count': 0,
   'negative_examples_count': 0,
   'score': 14.285714285714285,
   'relative_improvement': '28.57142857142856%',
   'token_len': 550.9473684210526},
  {'iteration': 2,
   'prompt': "You need to justify it in no more than 50 words when you answer 'yes' or 'no'.",
   'edit': 'reword_prompt',
   'positive_examples_count': 0,
   'negative_examples_count': 0,
   'score': 25.0,
   'relative_improvement': '75.00000000000003%',
   'token_len': 546.9473684210526},
  {'iteration': 3,
   'prompt': "When you answer 'yes' or 'no', you need to justify

In [19]:
unfairness_categories = ['A', 'CH', 'CR', 'J', 'LAW', 'LTD', 'TER', 'USE']
for cat in unfairness_categories:
    print("Category : ", cat)
    print(optimized_prompt_dict[cat][0]['score'])

In [21]:
import pickle
with open('optimized_prompt_dict_new_metaprompt.pickle', 'wb') as handle:
    pickle.dump(optimized_prompt_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [16]:
optimized_prompt_dict["USE"][0]['iteration']

0

In [13]:
{'LAW': [{'iteration': 0, 'prompt': "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words.", 'edit': 'inital - no edit', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 33.33333333333333}, {'iteration': 1, 'prompt': 'You should start your answer with "yes" or "no" and then justify your response in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 36.36363636363637, 'relative_improvement': '9.090909090909104%', 'token_len': 548.9473684210526}, {'iteration': 2, 'prompt': 'You should start your answer with "yes" or "no" and then justify your response in 50 words or less.', 'edit': 'add_positive_example', 'positive_examples_count': 1, 'negative_examples_count': 0, 'score': 33.33333333333333, 'relative_improvement': '-8.333333333333359%', 'token_len': 737.9473684210526}, {'iteration': 3, 'prompt': 'You should start your answer with "yes" or "no" and then justify your response in 50 words or less.', 'edit': 'add_negative_example', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 30.76923076923077, 'relative_improvement': '-7.692307692307676%', 'token_len': 784.9473684210526}, {'iteration': 4, 'prompt': 'You should justify your response in 50 words or less if you start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '62.5%', 'token_len': 782.9473684210526}, {'iteration': 5, 'prompt': ' - Justify response in 50 words or less\n - Start with "yes" or "no"', 'edit': 'to_bulletpoints', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 28.57142857142857, 'relative_improvement': '-42.85714285714286%', 'token_len': 778.9473684210526}, {'iteration': 6, 'prompt': 'In 50 words or less, start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 26.666666666666668, 'relative_improvement': '-6.666666666666654%', 'token_len': 776.9473684210526}, {'iteration': 7, 'prompt': 'Start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': '-100.0%', 'token_len': 770.9473684210526}, {'iteration': 8, 'prompt': 'Start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 770.9473684210526}, {'iteration': 9, 'prompt': 'Start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 770.9473684210526}, {'iteration': 10, 'prompt': 'Start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 770.9473684210526}, {'iteration': 11, 'prompt': 'Start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 770.9473684210526}, {'iteration': 12, 'prompt': 'Start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 770.9473684210526}, {'iteration': 13, 'prompt': 'Start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 770.9473684210526}, {'iteration': 14, 'prompt': 'Start with either "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 771.9473684210526}, {'iteration': 15, 'prompt': 'Start with either "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 771.9473684210526}, {'iteration': 16, 'prompt': 'Start with either "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 771.9473684210526}, {'iteration': 17, 'prompt': 'Start with either "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 771.9473684210526}, {'iteration': 18, 'prompt': 'Start with either "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 771.9473684210526}, {'iteration': 19, 'prompt': 'Start with either "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 771.9473684210526}, {'iteration': 20, 'prompt': 'Start with either "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': 'nan%', 'token_len': 771.9473684210526}], 'LTD': [{'iteration': 0, 'prompt': "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words.", 'edit': 'inital - no edit', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 50.0}, {'iteration': 1, 'prompt': 'Start your answer with "yes" or "no" and then justify it in no more than 50 words.', 'edit': 'reword_prompt', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 542.9473684210526}, {'iteration': 2, 'prompt': 'Start your answer with "yes" or "no" and then justify it in no more than 50 words.', 'edit': 'add_positive_example', 'positive_examples_count': 1, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 586.9473684210526}, {'iteration': 3, 'prompt': 'Start your answer with "yes" or "no" and then justify it in no more than 50 words.', 'edit': 'add_negative_example', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 57.14285714285714, 'relative_improvement': '14.28571428571428%', 'token_len': 620.9473684210526}, {'iteration': 4, 'prompt': 'Begin your answer with "yes" or "no" and then explain it in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 40.0, 'relative_improvement': '-29.999999999999993%', 'token_len': 619.9473684210526}, {'iteration': 5, 'prompt': ' - Answer with "yes" or "no"\n - Explain in 50 words or less', 'edit': 'to_bulletpoints', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 36.36363636363637, 'relative_improvement': '-9.090909090909083%', 'token_len': 615.9473684210526}, {'iteration': 6, 'prompt': 'Explain in 50 words or less if you have "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 57.14285714285714, 'relative_improvement': '57.14285714285712%', 'token_len': 615.9473684210526}, {'iteration': 7, 'prompt': 'Explain in 50 words or less if you have "yes" or "no".', 'edit': 'add_positive_example', 'positive_examples_count': 2, 'negative_examples_count': 1, 'score': 57.14285714285714, 'relative_improvement': '0.0%', 'token_len': 653.9473684210526}, {'iteration': 8, 'prompt': 'Explain in 50 words or less if you have "yes" or "no".', 'edit': 'add_positive_example', 'positive_examples_count': 3, 'negative_examples_count': 1, 'score': 36.36363636363637, 'relative_improvement': '-36.36363636363635%', 'token_len': 699.9473684210526}, {'iteration': 9, 'prompt': 'If you have "yes" or "no", explain in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 3, 'negative_examples_count': 1, 'score': 40.0, 'relative_improvement': '9.999999999999986%', 'token_len': 700.9473684210526}, {'iteration': 10, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 3, 'negative_examples_count': 1, 'score': 66.66666666666666, 'relative_improvement': '66.66666666666666%', 'token_len': 696.9473684210526}, {'iteration': 11, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 3, 'negative_examples_count': 1, 'score': 66.66666666666666, 'relative_improvement': '0.0%', 'token_len': 696.9473684210526}, {'iteration': 12, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'add_positive_example', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '-24.99999999999999%', 'token_len': 813.9473684210526}, {'iteration': 13, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}, {'iteration': 14, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}, {'iteration': 15, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}, {'iteration': 16, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}, {'iteration': 17, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}, {'iteration': 18, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}, {'iteration': 19, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}, {'iteration': 20, 'prompt': 'Explain "yes" or "no" in 50 words or less.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 813.9473684210526}], 'TER': [{'iteration': 0, 'prompt': "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words.", 'edit': 'inital - no edit', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 50.0}, {'iteration': 1, 'prompt': 'If you want to justify your response, start it with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 66.66666666666666, 'relative_improvement': '33.3333333333333%', 'token_len': 548.9473684210526}, {'iteration': 2, 'prompt': ' - Start with "yes" or "no" if you want to justify your response.', 'edit': 'to_bulletpoints', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '-24.99999999999999%', 'token_len': 547.9473684210526}, {'iteration': 3, 'prompt': ' - Start with "yes" or "no" if you want to justify your response.', 'edit': 'add_positive_example', 'positive_examples_count': 1, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 588.9473684210526}, {'iteration': 4, 'prompt': ' - Start with "yes" or "no" if you want to justify your response.', 'edit': 'add_negative_example', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 623.9473684210526}, {'iteration': 5, 'prompt': 'If you want to justify your response, start with "yes" or "no."', 'edit': 'reword_prompt', 'positive_examples_count': 1, 'negative_examples_count': 1, 'score': 66.66666666666666, 'relative_improvement': '33.3333333333333%', 'token_len': 623.9473684210526}, {'iteration': 6, 'prompt': 'If you want to justify your response, start with "yes" or "no."', 'edit': 'add_positive_example', 'positive_examples_count': 2, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '-24.99999999999999%', 'token_len': 676.9473684210526}, {'iteration': 7, 'prompt': 'If you want to justify your response, start with "yes" or "no."', 'edit': 'add_positive_example', 'positive_examples_count': 3, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 696.9473684210526}, {'iteration': 8, 'prompt': 'If you want to justify your response, start with "yes" or "no."', 'edit': 'add_positive_example', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 716.9473684210526}, {'iteration': 9, 'prompt': 'Start with "yes" or "no" to justify your response.', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 0.0, 'relative_improvement': '-100.0%', 'token_len': 712.9473684210526}, {'iteration': 10, 'prompt': 'To justify your response, start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': 'inf%', 'token_len': 713.9473684210526}, {'iteration': 11, 'prompt': 'To justify your response, start with "yes" or "no".', 'edit': 'add_positive_example', 'positive_examples_count': 5, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 827.9473684210526}, {'iteration': 12, 'prompt': 'To justify your response, start with "yes" or "no".', 'edit': 'reword_prompt', 'positive_examples_count': 5, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 827.9473684210526}, {'iteration': 13, 'prompt': 'To justify your response, start with "yes" or "no".', 'edit': 'add_positive_example', 'positive_examples_count': 6, 'negative_examples_count': 1, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 874.9473684210526}]}

{'LAW': [{'iteration': 0,
   'prompt': "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words.",
   'edit': 'inital - no edit',
   'positive_examples_count': 0,
   'negative_examples_count': 0,
   'score': 33.33333333333333},
  {'iteration': 1,
   'prompt': 'You should start your answer with "yes" or "no" and then justify your response in 50 words or less.',
   'edit': 'reword_prompt',
   'positive_examples_count': 0,
   'negative_examples_count': 0,
   'score': 36.36363636363637,
   'relative_improvement': '9.090909090909104%',
   'token_len': 548.9473684210526},
  {'iteration': 2,
   'prompt': 'You should start your answer with "yes" or "no" and then justify your response in 50 words or less.',
   'edit': 'add_positive_example',
   'positive_examples_count': 1,
   'negative_examples_count': 0,
   'score': 33.33333333333333,
   'relative_improvement': '-8.333333333333359%',
   'token_len': 737.9473684210526},
  {'iteration': 3,
   'prompt': 'You 

TER

In [13]:
{'TER': [{'iteration': 0, 'prompt': "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words.", 'edit': 'inital - no edit', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 50.0}, {'iteration': 1, 'prompt': "Start your answer with 'yes' or 'no' and then justify it in no more than 50 words.", 'edit': 'reword_prompt', 'positive_examples_count': 0, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 549.9473684210526}, {'iteration': 2, 'prompt': "Start your answer with 'yes' or 'no' and then justify it in no more than 50 words.", 'edit': 'add_positive_example', 'positive_examples_count': 1, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 591.9473684210526}, {'iteration': 3, 'prompt': "Start your answer with 'yes' or 'no' and then justify it in no more than 50 words.", 'edit': 'add_positive_example', 'positive_examples_count': 2, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 628.9473684210526}, {'iteration': 4, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'to_bulletpoints', 'positive_examples_count': 2, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 625.9473684210526}, {'iteration': 5, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_positive_example', 'positive_examples_count': 3, 'negative_examples_count': 0, 'score': 50.0, 'relative_improvement': '0.0%', 'token_len': 694.9473684210526}, {'iteration': 6, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_negative_example', 'positive_examples_count': 3, 'negative_examples_count': 1, 'score': 66.66666666666666, 'relative_improvement': '33.3333333333333%', 'token_len': 756.9473684210526}, {'iteration': 7, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_positive_example', 'positive_examples_count': 4, 'negative_examples_count': 1, 'score': 40.0, 'relative_improvement': '-39.99999999999999%', 'token_len': 786.9473684210526}, {'iteration': 8, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_negative_example', 'positive_examples_count': 4, 'negative_examples_count': 2, 'score': 57.14285714285714, 'relative_improvement': '42.85714285714284%', 'token_len': 798.9473684210526}, {'iteration': 9, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_positive_example', 'positive_examples_count': 5, 'negative_examples_count': 2, 'score': 40.0, 'relative_improvement': '-29.999999999999993%', 'token_len': 818.9473684210526}, {'iteration': 10, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'remove_positive_example', 'positive_examples_count': 4, 'negative_examples_count': 2, 'score': 33.33333333333333, 'relative_improvement': '-16.666666666666675%', 'token_len': 786.9473684210526}, {'iteration': 11, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'remove_positive_example', 'positive_examples_count': 3, 'negative_examples_count': 2, 'score': 66.66666666666666, 'relative_improvement': '100.0%', 'token_len': 749.9473684210526}, {'iteration': 12, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'remove_positive_example', 'positive_examples_count': 2, 'negative_examples_count': 2, 'score': 66.66666666666666, 'relative_improvement': '0.0%', 'token_len': 680.9473684210526}, {'iteration': 13, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_negative_example', 'positive_examples_count': 2, 'negative_examples_count': 3, 'score': 57.14285714285714, 'relative_improvement': '-14.28571428571428%', 'token_len': 719.9473684210526}, {'iteration': 14, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_negative_example', 'positive_examples_count': 2, 'negative_examples_count': 4, 'score': 66.66666666666666, 'relative_improvement': '16.66666666666665%', 'token_len': 733.9473684210526}, {'iteration': 15, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_negative_example', 'positive_examples_count': 2, 'negative_examples_count': 5, 'score': 57.14285714285714, 'relative_improvement': '-14.28571428571428%', 'token_len': 750.9473684210526}, {'iteration': 16, 'prompt': " - Start with 'yes' or 'no' and justify in no more than 50 words.", 'edit': 'add_negative_example', 'positive_examples_count': 2, 'negative_examples_count': 6, 'score': 57.14285714285714, 'relative_improvement': '0.0%', 'token_len': 767.9473684210526}, {'iteration': 17, 'prompt': "Start with 'yes' or 'no' and justify in 50 words or less.", 'edit': 'reword_prompt', 'positive_examples_count': 2, 'negative_examples_count': 6, 'score': 80.0, 'relative_improvement': '40.000000000000014%', 'token_len': 765.9473684210526}, {'iteration': 18, 'prompt': "If you want to justify in 50 words or less, start with 'yes' or 'no'.", 'edit': 'reword_prompt', 'positive_examples_count': 2, 'negative_examples_count': 6, 'score': 57.14285714285714, 'relative_improvement': '-28.57142857142858%', 'token_len': 769.9473684210526}, {'iteration': 19, 'prompt': "If you want to justify in 50 words or less, start with 'yes' or 'no'.", 'edit': 'reword_prompt', 'positive_examples_count': 2, 'negative_examples_count': 6, 'score': 57.14285714285714, 'relative_improvement': '0.0%', 'token_len': 769.9473684210526}, {'iteration': 20, 'prompt': "If you want to justify in 50 words or less, start with 'yes' or 'no'.", 'edit': 'reword_prompt', 'positive_examples_count': 2, 'negative_examples_count': 6, 'score': 57.14285714285714, 'relative_improvement': '0.0%', 'token_len': 769.9473684210526}]}

{'TER': [{'iteration': 0,
   'prompt': "Start your answer with 'yes' or 'no' and then justify your response in no more than 50 words.",
   'edit': 'inital - no edit',
   'positive_examples_count': 0,
   'negative_examples_count': 0,
   'score': 50.0},
  {'iteration': 1,
   'prompt': "Start your answer with 'yes' or 'no' and then justify it in no more than 50 words.",
   'edit': 'reword_prompt',
   'positive_examples_count': 0,
   'negative_examples_count': 0,
   'score': 50.0,
   'relative_improvement': '0.0%',
   'token_len': 549.9473684210526},
  {'iteration': 2,
   'prompt': "Start your answer with 'yes' or 'no' and then justify it in no more than 50 words.",
   'edit': 'add_positive_example',
   'positive_examples_count': 1,
   'negative_examples_count': 0,
   'score': 50.0,
   'relative_improvement': '0.0%',
   'token_len': 591.9473684210526},
  {'iteration': 3,
   'prompt': "Start your answer with 'yes' or 'no' and then justify it in no more than 50 words.",
   'edit': 'add_posit

In [ ]:
USE